# モーターのテスト

下はライブラリをインポートする処理

In [42]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import smbus
import time
import sys

bus = smbus.SMBus(1)

関数の中身です．

In [48]:
#スピードは1~58(0x3A)の範囲で指定
def L_forward(speed): 
    # 左モーターの正転
    cmd = 0x01
    sval = cmd | ((speed+5)<<2)
    bus.write_i2c_block_data(0x64,0x00,[sval])
    
def L_back(speed):
    #左モーターの逆転
    cmd = 0x02
    sval = cmd | ((speed+5)<<2)
    bus.write_i2c_block_data(0x64,0x00,[sval])
    
def L_stop():
    #左モーターの停止
    bus.write_i2c_block_data(0x64,0x00,[0x00])
    
def R_forward(speed):
    #右モーターの正転
    cmd = 0x01
    sval = cmd | ((speed+5)<<2)
    bus.write_i2c_block_data(0x62,0x00,[sval])
    
def R_back(speed):
    #右モーターの逆転
    cmd = 0x02
    sval = cmd | ((speed+5)<<2)
    bus.write_i2c_block_data(0x62,0x00,[sval])    
    
def R_stop():
    #右モーターのストップ
    bus.write_i2c_block_data(0x62,0x00,[0x00])
    
def Turn_L(speed):
    #左に曲がる処理
    #右のほうがspeedが左よりも10進数で15だけ大きい
    #ちなみにspeedが15よりも小さい時はspeedの半分の速さで動く
    
    left = speed - 0xF
    if left  < 0:
        left = speed / 0x2
        
    L_forward(left)
    R_forward(speed)
    
def Turn_R(speed):
    #右に曲がる処理
    #左のほうがspeedが左よりも10進数で15だけ大きい
    #ちなみにspeedが15よりも小さい時はspeedの半分の速さで動く
    
    right = speed - 0xF
    if right  < 0:
        right = speed / 0x2
        
    L_forward(speed)
    R_forward(right)
    
def Turn_there(speed):
    #　その場で回転することができる．
    # できれば低速の方が好ましい
    
    L_forward(speed)
    R_back(speed)

## まずは両方のモーターの正転から始める．
forループで速さを逐次変化させ，だんだんと速さを早くしていく．暴走する可能性もあるのでキルスイッチ（物理スイッチまたは下にある処理）を
設置し，暴走状態になった時でも停止ができるようにするとよい．

In [28]:
for i in range(0x01,0x1E,1):
    try:
        L_forward(i)
        R_forward(i)
    except:
        pass
    time.sleep(0.1)
    
L_stop()
R_stop()


## 両方のモーターのバックをする
これも正転と同じようにforループで速さを逐次変化させている．
バックは念のためということで速さを抑えたほうがいいと思われる．

In [50]:
for i in range(0x01,0x14,1):
    try:
        L_back(i)
        R_back(i)
    except:
        pass
    time.sleep(0.1)
L_stop()
R_stop()

## 左右で別の動きをさせる

### その場で回転
今回のキモである左右別々の動きである．これを実装するとその場での回転ができる．速すぎるとパーツが脱落する恐れもあり，あまり推奨できないため低速で回転することがいいと思われる．

In [53]:
Turn_there(0xF)
time.sleep(2)

L_stop()
R_stop()

### 右方向に曲がる
右方向に舵角を取り，右方向に曲がる処理である．

In [41]:
Turn_R(0x14)
time.sleep(2)

L_stop()
R_stop()

### 左方向に曲がる
左方向に高くを取り，左方向に曲がる処理である．

In [54]:
Turn_L(0x14)
time.sleep(2)

L_stop()
R_stop()

# キルスイッチ
突然動きが止まらなくなったり，強制的に動きを停止させる時にキルスイッチの役目をする処理，ただstopさせているだけだがこれで停止できる．

In [ ]:
L_stop()
R_stop()